In [1]:
%load_ext autoreload
%autoreload 2

import os
import torch

import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib

model_name='Llama2-7b-hf'
#model_name='Llama-3.2-1B'

DTYPE = torch.float16
fp_model, _ = qlib.load_llama(
	model_name=model_name, 
	device_map='cpu', 
	torch_dtype=DTYPE
)

qmodel = qlib.QuantizedLlamaForCausalLM.from_pretrained(
	f'/home/msst/repo/Quantization/ml/llm/pretrained_models/Llama2-7b-trellis',
	torch_dtype=torch.float16,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LOL, Im custom!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/msst/repo/Quantization/ml/llm/pretrained_models/Llama2-7b-trellis were not used when initializing QuantizedLlamaForCausalLM: {'model.layers.11.mlp.gate_proj.weight', 'model.layers.23.self_attn.q_proj.weight', 'model.layers.18.self_attn.v_proj.weight', 'model.layers.15.mlp.up_proj.weight', 'model.layers.27.self_attn.v_proj.weight', 'model.layers.10.self_attn.q_proj.weight', 'model.layers.26.mlp.up_proj.weight', 'model.layers.12.self_attn.v_proj.weight', 'model.layers.3.self_attn.o_proj.weight', 'model.layers.24.self_attn.k_proj.weight', 'model.layers.10.self_attn.k_proj.weight', 'model.layers.9.mlp.down_proj.weight', 'model.layers.19.mlp.up_proj.weight', 'model.layers.20.self_attn.q_proj.weight', 'model.layers.3.mlp.gate_proj.weight', 'model.layers.11.self_attn.k_proj.weight', 'model.layers.4.self_attn.k_proj.weight', 'model.layers.14.self_attn.o_proj.weight', 'model.layers.1.self_attn.q_proj.weight', 'model.layers.23.mlp.down_proj.weight', 

In [2]:
for module_name, qmodule in qmodel.named_modules():
    if isinstance(qmodule, qlib.TrellisLinear):
        qmodule.wrap_module(
            fp_model.get_submodule(module_name), 
            verbose=True,
            module_name=module_name)



/home/msst/miniconda3/envs/qenv/lib/python3.10/site-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(


model.layers.0.self_attn.q_proj


100%|██████████| 256/256 [00:19<00:00, 12.91it/s]


error: 0.078 ± 0.099
model.layers.0.self_attn.k_proj


100%|██████████| 256/256 [00:19<00:00, 12.87it/s]


error: 0.078 ± 0.076
model.layers.0.self_attn.v_proj


 16%|█▋        | 42/256 [00:03<00:15, 13.40it/s]


KeyboardInterrupt: 

In [3]:
tlp = qmodel.config.trellis_linear_params
checkpoints_path = '/home/msst/repo/Quantization/logs/checkpoints_Llama2-7b-hf/trellis/'

qmodel.save_pretrained(
	os.path.join(
		checkpoints_path,
		"test_mx"
	)
)
